## Import libs

In [1]:
import pandas as pd
import numpy as np
import os 

import seaborn as sns
import matplotlib.pyplot as plt

import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
mlflow.set_experiment("random-forest-hyperopt")

<Experiment: artifact_location='mlflow-artifacts:/161295597375028244', creation_time=1746883127017, experiment_id='161295597375028244', last_update_time=1746883127017, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>

## Download data

In [4]:
!python3 preprocess_data.py --raw_data_path "./data" --dest_path "./output"

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

**What's the version that you have?**

    ✅ 2.17.2

In [5]:
!mlflow --version

mlflow, version 2.17.2


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

**How many files were saved to `OUTPUT_FOLDER`?**

    ✅ 4

In [6]:
!python preprocess_data.py --raw_data_path "./data" --dest_path "./output"

In [7]:
!ls -lh ./output

total 17088
-rw-r--r--  1 kseniialakhman  staff   128K May 10 16:58 dv.pkl
-rw-r--r--  1 kseniialakhman  staff   2.3M May 10 16:58 test.pkl
-rw-r--r--  1 kseniialakhman  staff   2.3M May 10 16:58 train.pkl
-rw-r--r--  1 kseniialakhman  staff   2.1M May 10 16:58 val.pkl


In [8]:
!ls -1q "./output" | wc -l

       4


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [`homework`](homework). 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

**What is the value of the `min_samples_split` parameter:**

    ✅ 2


In [9]:
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

In [13]:
def run_train(data_path: str):
    mlflow.sklearn.autolog()
    
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))
    
    with mlflow.start_run():
        rf = RandomForestRegressor(max_depth=10, random_state=0)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)

        with open('models/rand_forest.bin', 'wb+') as f_out:
            pickle.dump(rf, f_out)
        
        mlflow.log_artifact(local_path="models/rand_forest.bin", artifact_path="models_pickle")


In [15]:
run_train('./output')

2025/05/10 17:08:19 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2025/05/10 17:08:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run salty-grouse-296 at: http://127.0.0.1:5000/#/experiments/161295597375028244/runs/b24088df11374895bfdb772395bd8e18.
2025/05/10 17:08:51 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/161295597375028244.


## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

**In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?**

    ✅ `default-artifact-root`

Command:

```mlflow server --backend-store-uri sqlite:///mlflow.sqlite --default-artifact-root ./artifacts```

In [17]:
mlflow.set_tracking_uri('sqlite:///mlflow.sqlite')
mlflow.set_experiment("random-forest-hyperopt")

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/05/10 21:08:28 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/kseniialakhman/projects/mlops/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1746900508290, experiment_id='1', last_update_time=1746900508290, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>

## Question 5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

**What's the best validation RMSE that you got?**

    ✅ 5.335

In [19]:
!python3 hpo.py

  0%|                                    | 0/15 [00:00<?, ?trial/s, best loss=?]2025/05/10 21:14:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run nebulous-fish-821 at: http://127.0.0.1:5000/#/experiments/1/runs/1b3bc014a5eb42fdb76cf66b7b852215.

2025/05/10 21:14:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.

  7%|▋          | 1/15 [00:15<03:38, 15.62s/trial, best loss: 5.370086069268862]2025/05/10 21:14:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run glamorous-lamb-183 at: http://127.0.0.1:5000/#/experiments/1/runs/5402b6fc62474c42949756865f029859.

2025/05/10 21:14:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.

 13%|█▍         | 2/15 [00:16<01:30,  6.96s/trial, best loss: 5.370086069268862]2025/05/10 21:14:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run unique-snipe-466 at: http://127.0.0.1:5000/#/experiments/1/runs/506

best loss: 5.335419588556921

## Question 6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

    ✅ 5.567
    

In [35]:
!python3 register_model.py

2025/05/10 21:35:21 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2025/05/10 21:35:46 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2025/05/10 21:36:21 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2025/05/10 21:36:46 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2025/05/10 21:37:16 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
Registered model 'random-forest-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'random-forest-regressor'.


Best test_rmse is 5.567408012462019